# Systematic studies with stopping protons

In [1]:
%load_ext autoreload
%autoreload 2

import math
from itertools import product
import uproot
import awkward
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.stats import binned_statistic

import matplotlib
matplotlib.rcParams.update({'font.size': 12})
from calo_likelihood import caloLikelihood
from general_functions import *
from loaders import *
from plotter import *
from categorisation import *
from initializers import *

In [2]:
# today = 'calorimetry'
today = 'calorimetry_trial2/protons_mod'

In [3]:
branches = [
    'run',
    'sub',
    'evt',
    
    'backtracked_pdg',
    'backtracked_e',
    'backtracked_purity',
    'backtracked_completeness',
    'backtracked_overlay_purity',
    'backtracked_end_process',
    
    'backtracked_start_x',
    'backtracked_start_y',
    'backtracked_start_z',
    
    'backtracked_sce_start_x',
    'backtracked_sce_start_y',
    'backtracked_sce_start_z',
    
    'nplanehits_U',
    'nplanehits_V',
    'nplanehits_Y',
    'trk_score',

    'generation',
    'trk_daughters',
    'shr_daughters',

    'trk_sce_start_x',
    'trk_sce_start_y',
    'trk_sce_start_z',

    'trk_sce_end_x',
    'trk_sce_end_y',
    'trk_sce_end_z',
    
    'trk_theta',
    'trk_phi',
     
    'trk_dir_x',
    'trk_dir_y',
    'trk_dir_z',

    'trk_len',
    'trk_distance',
    'longest',
    
    'trk_mcs_muon_mom',
    'trk_range_muon_mom',
    
    'is_hit_montecarlo_u',
    'is_hit_montecarlo_v',
    'is_hit_montecarlo_y',
    
    'dqdx_u',
    'dqdx_v',
    'dqdx_y',

    'dedx_u',
    'dedx_v',
    'dedx_y',

    'rr_u',
    'rr_v',
    'rr_y',

    'pitch_u',
    'pitch_v',
    'pitch_y',
    
    'dir_x_u',
    'dir_x_v',
    'dir_x_y',
    
    'dir_y_u',
    'dir_y_v',
    'dir_y_y',
    
    'dir_z_u',
    'dir_z_v',
    'dir_z_y',
]

In [4]:
directory = '/home/nic/Desktop/MicroBooNE/calorimetry_likelihood/'
pot_beam_on = 4.054e+19
n_triggers_on = 8977290.0
n_triggers_off = 33240727.0

fraction = 1
pot_beam_on *= fraction

file_lists = {}

file_lists['beam_on'] = directory+'v07/beam_on/out_proton_skimmed.root'
file_lists['beam_off'] = directory+'v07/beam_off/out_proton_skimmed.root'
file_lists['bnb_nu_mod'] = directory+'v08/bnb_nu_mod/out_proton_skimmed.root'
file_lists['bnb_nu_recomb_mod'] = directory+'v08/recomb_mod/out_proton_skimmed.root'
file_lists['bnb_nu_wire_dedx_mod'] = directory+'v08/wire_mod_dedx/out_proton_skimmed.root'
file_lists['bnb_dirt'] = directory+'v07/bnb_dirt/out_proton_skimmed.root'

folder = "nuselection"
events_tree_name = "CalorimetryAnalyzer"
pot_tree_name = "SubRun"

In [5]:
arrays, pot = load_data_calo(file_lists, 
                        branches, 
                        folder, 
                        events_tree_name, 
                        pot_tree_name, 
                        labels=['beam_on', 'beam_off', 'bnb_nu_mod', 'bnb_nu_recomb_mod', 'bnb_nu_wire_dedx_mod', 'bnb_dirt'],
                        lazy=False,
                        fraction=fraction)

start loading beam_on
start loading beam_off
start loading bnb_nu_mod
start loading bnb_nu_recomb_mod
start loading bnb_nu_wire_dedx_mod
start loading bnb_dirt
Done!


In [6]:
scale_factors = compute_scale_factors(pot, pot_beam_on, n_triggers_on, n_triggers_off)

In [7]:
scale_factors

{'beam_on': 1,
 'beam_off': 0.27006900300345416,
 'bnb_nu_mod': 0.380675079365475,
 'bnb_nu_recomb_mod': 0.3947884676946471,
 'bnb_nu_wire_dedx_mod': 0.37173495568281745,
 'bnb_dirt': 0.12607623142910898}

In [8]:
for name, array in arrays.items():
    print(name)
    initialisation_for_caloriemtry_data_mc(array)
    compute_pid(array)

beam_on


/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/general_functions.py:33: RuntimeWarning: divide by zero encountered in true_divide
  return 0.3 / cos


beam_off
bnb_nu_mod
bnb_nu_recomb_mod
bnb_nu_wire_dedx_mod


/home/nic/.local/lib/python3.7/site-packages/awkward/array/jagged.py:1035: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


bnb_dirt


In [9]:
for name, array in arrays.items():
    add_norm_variable(array, 'llr_012')

In [10]:
my_plotter = plotter(arrays, scale_factors, pot_beam_on, branch_weights_name='weightSplineTimesTune')

In [11]:
bnb_nu_sist_samples = ['bnb_nu_mod', 'bnb_nu_recomb_mod', 'bnb_nu_wire_dedx_mod']
bnb_nu_sist_labels = ['Central Value', 'Recombination mod', 'Wire dE/dx mod']

# Base plots data/MC comparison 

In [12]:
variables = [
#     'trk_len',
#     'trk_score',
#     'trk_theta',
#     'trk_phi',
    'llr_012_n'
]

binnings = {
    'trk_len': [50, 0, 600],
    'trk_score': [50, 0, 1],
    'trk_theta': [50, 0, np.pi],
    'trk_phi': [50, -np.pi, np.pi],
    'llr_012_n': [50, -1, 1],
}

xlabels = {
    'trk_len': 'Track length [cm]',
    'trk_score': r'Track score',
    'trk_theta': r'Track $\theta$',
    'trk_phi': r'Track $\phi$',
    'llr_012_n': 'Track PID',
}

log = {
    'trk_len': True,
    'trk_score': False,
    'trk_theta': False,
    'trk_phi': False,
    'llr_012_n': False,
}

In [13]:
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/base_plots/
save_folder_base = '/home/nic/Dropbox/MicroBooNE/_fig/' + today + '/base_plots/'

title_base = 'Trk-score > 0.5\ntrk-vtx distance < 5 cm, start-end fiducial'
for var in variables:
    for sample, label in zip(bnb_nu_sist_samples, bnb_nu_sist_labels):
        title = label + '\n' + title_base
        out = my_plotter.plot_pot_normalised_var(variable=var,
                                       binning=binnings[var], 
                                       categories=pdg_categories, 
                                       additional_selection=None, 
                                       prediction_datasets=['bnb_dirt', sample],
                                       xlabel=xlabels[var], 
                                       title=title,
                                       log=log[var], 
                                       subtracted=False, 
                                       onebin=False,
                                       legend=False)
        plt.savefig(save_folder_base + var + '_before_pid_cut_pot_{}.png'.format(sample), dpi=300)
        plt.savefig(save_folder_base + var + '_before_pid_cut_pot_{}.pdf'.format(sample))
        plt.close()


        out = my_plotter.plot_pot_normalised_var(variable=var,
                                       binning=binnings[var], 
                                       categories=pdg_categories, 
                                       additional_selection=None, 
                                       prediction_datasets=['bnb_dirt', sample],
                                       xlabel=xlabels[var], 
                                       title=title,
                                       log=log[var], 
                                       subtracted=True, 
                                       onebin=False,
                                       area_norm=True,
                                       legend=False)
        plt.savefig(save_folder_base + var + '_before_pid_cut_area_norm_{}.png'.format(sample), dpi=300)
        plt.savefig(save_folder_base + var + '_before_pid_cut_area_norm_{}.pdf'.format(sample))
        plt.close()

/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:184: RuntimeWarning: invalid value encountered in true_divide
  bin_relative_errors_data = beam_on_y_err / beam_on_values
/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:184: RuntimeWarning: invalid value encountered in true_divide
  bin_relative_errors_data = beam_on_y_err / beam_on_values
/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:184: RuntimeWarning: invalid value encountered in true_divide
  bin_relative_errors_data = beam_on_y_err / beam_on_values
/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:184: RuntimeWarning: invalid value encountered in true_divide
  bin_relative_errors_data = beam_on_y_err / beam_on_values
/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:184: RuntimeWarning: invalid value encountered in true_divide
  bin_relative_errors_data = beam_on_y_err / beam_on_values
/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:184: RuntimeWarning: inv

# Plots at low rr, in bins of rr and pitch

In [14]:
this_folder = 'low_rr'
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/$this_folder/plane0/
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/$this_folder/plane1/
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/$this_folder/plane2/
this_save_folder = '/home/nic/Dropbox/MicroBooNE/_fig/' + today + '/{}/'.format(this_folder)

parameters = ['rr', 'pitch']
parameters_legend_names = ['residual range', 'pitch']
parameters_bin_edges = [[0, 2, 5, 10, 20, 30, 50, 150, 300], [0.3, 0.4, 1, 30]]
dedx_vars = 'dqdx'

binning = [60, 0, 1500]
title_base = 'Trk-score > 0.5, track PID < -0.1\ntrk-vtx distance < 5 cm, start-end fiducial'

chi2_tables = {}

for sample, label in zip(bnb_nu_sist_samples, bnb_nu_sist_labels):
    chi2_tables[sample] = {}
    for plane_num, plane in zip([2, 1, 0], ['_y', '_v', '_u']):
        chi2_tables[sample][plane_num] = []
        for par0_edges in zip(parameters_bin_edges[0][:-1], parameters_bin_edges[0][1:]):
            chi2_tables[sample][plane_num].append([])
            for par1_edges in zip(parameters_bin_edges[1][:-1], parameters_bin_edges[1][1:]):
                this_parameters = [par+plane for par in parameters]
                def sel_function(array):
                    return (array['first_last_hit_mask'+plane]) &\
                           (array['llr_012_n'] < -0.1) &\
                            ((array[this_parameters[0]]) > par0_edges[0]) &\
                            ((array[this_parameters[0]]) <= par0_edges[1]) &\
                            ((array[this_parameters[1]]) > par1_edges[0]) &\
                            ((array[this_parameters[1]]) <= par1_edges[1])
                title = label + '\n' + title_base + '\nplane {}, {:.2f} < {} < {:.2f} & {:.2f} < {} < {:.2f}'.format(plane_num, par0_edges[0], 
                                                             parameters_legend_names[0],
                                                             par0_edges[1],
                                                             par1_edges[0], 
                                                             parameters_legend_names[1],
                                                             par1_edges[1],)

                save_label = '{}_{:04.0f}_{}_{:04.0f}_{:04.1f}_{}_{:04.1f}'.format(sample, 
                                                             par0_edges[0], 
                                                             parameters[0],
                                                             par0_edges[1],
                                                             par1_edges[0], 
                                                             parameters[1],
                                                             par1_edges[1]).replace('.', '').replace('$', '').replace('\\', '')

                out = my_plotter.plot_pot_normalised_var(variable='dqdx'+plane+"_cali",
                                           binning=binning, 
                                           categories=pdg_categories, 
                                           additional_selection=sel_function, 
                                           prediction_datasets=['bnb_dirt', sample],
                                           xlabel=r'Calibrated Hit dQ/dx [ADC/cm]', 
                                           title=title,
                                           log=False,
                                           subtracted=True,
                                           area_norm=True,
                                           onebin=False)
                plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '.png', dpi=300)
                plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '.pdf')
                plt.close()

                chi2_tables[sample][plane_num][-1].append(out[1]/out[2])

                out = my_plotter.plot_pot_normalised_var(variable='dqdx'+plane+"_cali",
                                           binning=binning, 
                                           categories=pdg_categories, 
                                           additional_selection=sel_function, 
                                           prediction_datasets=['bnb_dirt', sample],
                                           xlabel=r'Calibrated Hit dQ/dx [ADC/cm]', 
                                           title=title,
                                           log=False,
                                           subtracted=True,
                                           area_norm=True,
                                           onebin=False,
                                           legend=False)
                plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_noleg.png', dpi=300)
                plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_noleg.pdf')
                plt.close()

/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:182: RuntimeWarning: divide by zero encountered in true_divide
  bin_contents_ratio = beam_on_values / bin_contents_prediction[-1]
/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:183: RuntimeWarning: invalid value encountered in true_divide
  bin_relative_errors_prediction = bin_errors_prediction / bin_contents_prediction[-1]
/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:184: RuntimeWarning: invalid value encountered in true_divide
  bin_relative_errors_data = beam_on_y_err / beam_on_values
/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:182: RuntimeWarning: divide by zero encountered in true_divide
  bin_contents_ratio = beam_on_values / bin_contents_prediction[-1]
/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:183: RuntimeWarning: invalid value encountered in true_divide
  bin_relative_errors_prediction = bin_errors_prediction / bin_contents_prediction[-1]
/home/nic/Dro

In [15]:
# summary plots of the chi2
plt.rcParams.update({'font.size': 10})

this_folder = 'low_rr'
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/$this_folder/
this_save_folder = '/home/nic/Dropbox/MicroBooNE/_fig/' + today + '/{}/'.format(this_folder)

parameters = ['rr', 'pitch']
parameters_legend_names = ['residual range', 'pitch']
parameters_bin_edges = [[0, 2, 5, 10, 20, 30, 50, 150, 300], [0.3, 0.4, 1, 30]]

def chi2_plot(table, title, save_name):
    plt.pcolormesh(table)
    plt.title(title, loc='left')
    plt.colorbar()
    plt.xticks(range(len(parameters_bin_edges[1])), ['{:.2g}'.format(p) for p in parameters_bin_edges[1]])
    plt.yticks(range(len(parameters_bin_edges[0])), ['{:.2g}'.format(p) for p in parameters_bin_edges[0]])
    plt.xlabel("pitch [cm]")
    plt.ylabel("residual range [cm]")
    
    for i in range(len(parameters_bin_edges[1])-1):
        for j in range(len(parameters_bin_edges[0])-1):
            text = plt.text(i+0.5, j+0.5, "{:.3g}".format(table[j, i]),
                           ha="center", va="center", color="k")
    plt.tight_layout()
    plt.savefig(save_name + '.png', dpi=300)
    plt.savefig(save_name + '.pdf')
    plt.close()

for plane_num in range(3):
    chi2_table_cv = np.array(chi2_tables['bnb_nu_mod'][plane_num])
    for sample, label in zip(bnb_nu_sist_samples, bnb_nu_sist_labels):
        this_title_base = " {}\n".format(label) + title_base + '\nplane {}\nMicroBooNE Preliminary'.format(plane_num)

        chi2_table = np.array(chi2_tables[sample][plane_num])

        chi2_plot(chi2_table, title=r"$X^2$" + this_title_base, 
                           save_name=this_save_folder + '{}_plane_{}_chi2'.format(sample, plane_num))
        
        if sample != 'bnb_nu_mod':
            chi2_plot(chi2_table-chi2_table_cv, title=r"$X^2_{MOD} - X^2_{CV}/ndof$" + this_title_base, 
                               save_name=this_save_folder + '{}_plane_{}_delta_chi2'.format(sample, plane_num))

            chi2_plot(chi2_table/chi2_table_cv, r"$X^2_{MOD}/X^2_{CV}$" + this_title_base, 
                               save_name=this_save_folder + '{}_plane_{}_chi2_ratio'.format(sample, plane_num))

# LLR PID in function of the phase space

## LLRPID in function of pitch and phi wrt collection plane

In [16]:
# this_folder = 'llr_pid'
# !mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/protons/$this_folder/
# this_save_folder = '/home/nic/Dropbox/MicroBooNE/_fig/' + today + '/protons/{}/'.format(this_folder)

# parameters = ['trk_pitch_y', 'trk_abs_theta_yx_y']
# parameters_legend_names = ['track pitch Y', r'track $\phi$']
# parameters_bin_edges = [np.array([0.3, 0.4, 0.7, 1., 1.5, 30]), np.linspace(0, np.pi/2, 7)]
# var = 'llr_012_n'

# binning = [25, -1, 1]
# title_base = 'Trk-score > 0.5\ntrk-vtx distance < 5 cm, start-end fiducial'

# for par0_edges in zip(parameters_bin_edges[0][:-1], parameters_bin_edges[0][1:]):
#     for par1_edges in zip(parameters_bin_edges[1][:-1], parameters_bin_edges[1][1:]):
#         def sel_function(array):
#             return ((array[parameters[0]]) > par0_edges[0]) &\
#                     ((array[parameters[0]]) <= par0_edges[1]) &\
#                     ((array[parameters[1]]) > par1_edges[0]) &\
#                     ((array[parameters[1]]) <= par1_edges[1])
#         title = title_base + '\n{:.2f} < {} < {:.2f} & {:.2f} < {} < {:.2f}'.format(par0_edges[0], 
#                                                      parameters_legend_names[0],
#                                                      par0_edges[1],
#                                                      par1_edges[0], 
#                                                      parameters_legend_names[1],
#                                                      par1_edges[1],)

#         save_label = '{:04.1f}_{}_{:04.1f}_{:04.2f}_{}_{:04.2f}'.format( 
#                                                      par0_edges[0], 
#                                                      parameters[0],
#                                                      par0_edges[1],
#                                                      par1_edges[0], 
#                                                      parameters[1],
#                                                      par1_edges[1]).replace('.', '').replace('$', '').replace('\\', '')

#         out = my_plotter.plot_pot_normalised_var(variable=var,
#                                        binning=binning, 
#                                        categories=pdg_categories, 
#                                        additional_selection=sel_function, 
#                                        prediction_datasets=['bnb_dirt', 'bnb_nu'],
#                                        xlabel='Track PID - 3 planes',
#                                        title=title,
#                                        log=False,
#                                        subtracted=False,
#                                        area_norm=False,
#                                        onebin=False)
#         plt.savefig(this_save_folder + save_label + '_pot.png', dpi=300)
#         plt.savefig(this_save_folder + save_label + '_pot.pdf')
#         plt.close()


#         out = my_plotter.plot_pot_normalised_var(variable=var,
#                                        binning=binning, 
#                                        categories=pdg_categories, 
#                                        additional_selection=sel_function, 
#                                        prediction_datasets=['bnb_dirt', 'bnb_nu'],
#                                        xlabel='Track PID - 3 planes',
#                                        title=title,
#                                        log=False,
#                                        subtracted=True,
#                                        area_norm=True,
#                                        onebin=False)
#         plt.savefig(this_save_folder + save_label + '_area_norm.png', dpi=300)
#         plt.savefig(this_save_folder + save_label + '_area_norm.pdf')
#         plt.close()

#         out2 = my_plotter.plot_pot_normalised_var(variable=var,
#                                        binning=binning, 
#                                        categories=pdg_categories, 
#                                        additional_selection=sel_function, 
#                                        prediction_datasets=['bnb_dirt', 'bnb_nu'],
#                                        xlabel='Track PID - 3 planes',
#                                        title=title,
#                                        log=False,
#                                        subtracted=False,
#                                        area_norm=False,
#                                        legend=False)
#         plt.savefig(this_save_folder + save_label + '_pot_noleg.png', dpi=300)
#         plt.savefig(this_save_folder + save_label + '_pot_noleg.pdf')
#         plt.close()


#         out2 = my_plotter.plot_pot_normalised_var(variable=var,
#                                        binning=binning, 
#                                        categories=pdg_categories, 
#                                        additional_selection=sel_function, 
#                                        prediction_datasets=['bnb_dirt', 'bnb_nu'],
#                                        xlabel='Track PID - 3 planes',
#                                        title=title,
#                                        log=False,
#                                        subtracted=True,
#                                        area_norm=True,
#                                        legend=False)
#         plt.savefig(this_save_folder + save_label + '_area_norm_noleg.png', dpi=300)
#         plt.savefig(this_save_folder + save_label + '_area_norm_noleg.pdf')
#         plt.close()